In [1]:
import pandas as pd
import glob
import re
import os 

prompts_paths = glob.glob(".\\prompts\\*\\*.txt")
essays_paths = glob.glob(".\\prompts\\*\\*\\*.txt")
marks_paths = glob.glob(".\\prompts\\*\\*\\*.csv")

df = pd.DataFrame(columns=['prompt', 'essay', 'mark1', 'mark2', 'mark3', 'mark4', 'mark5'])

for prompt_path in prompts_paths:
    prompt_number = re.search("(\d+)", prompt_path).group(0)
    with open(prompt_path, 'r') as f:
        prompt = f.read()
        find1 = re.findall("(.*?)Como enviar sua redação", prompt)
        find2 = re.findall("(.*?)ObservaçõesSeu", prompt)
        if find1:
            prompt = find1[0]
        if find2:
            prompt = find2[0]
    essays = []
    marks = pd.DataFrame({0: [], 1:[], 2:[], 3:[], 4:[]})
    for essay_path in essays_paths:
        #Get essays for this prompt, and the original one
        if "prompt"+str(prompt_number)+"\\" in essay_path and "original" in essay_path:
            with open(essay_path, "r", encoding='utf-8') as original:
                essay = original.read()
                essays.append(essay)
            mark_path = essay_path.replace("_original.txt", "_mark.csv")
            mark = pd.read_csv(mark_path).transpose().drop(axis=0, index="Topics").drop(columns=[5])
            marks = pd.concat([marks, mark], ignore_index=True)
    prompts = [prompt] * len(essays)
    sub_df = pd.DataFrame({'prompt': prompts, 'essay': essays, 'mark1': marks[0], 'mark2':marks[1], 'mark3':marks[2], 'mark4':marks[3], 'mark5':marks[4]})
    df = pd.concat([df, sub_df], ignore_index=True)
len(df)

456

In [2]:
def to_list(values):
    print(values.tolist())
df['marks'] = df[df.columns[2:7]].values.astype(int).tolist()
df

,prompt,essay,mark1,mark2,mark3,mark4,mark5,marks
0,O número de pessoas desempregadas no mundo dev...,Qualificações para o mercado de trabalho\n\nAn...,80,120,120,80,80,"[80, 120, 120, 80, 80]"
1,O número de pessoas desempregadas no mundo dev...,Futuro da crise do desemprego\n\nA Qualificaçã...,80,80,80,80,80,"[80, 80, 80, 80, 80]"
2,O número de pessoas desempregadas no mundo dev...,O progresso da tecnologia\n\nNo decorrer dos s...,160,160,160,160,120,"[160, 160, 160, 160, 120]"
3,O número de pessoas desempregadas no mundo dev...,O advento tecnológico \n\nAs constantes mudanç...,160,120,120,120,120,"[160, 120, 120, 120, 120]"
4,O número de pessoas desempregadas no mundo dev...,"Transformações laborais\n\nA noção de emprego,...",120,120,120,120,120,"[120, 120, 120, 120, 120]"
...,...,...,...,...,...,...,...,...
451,Reportagem publicada pelo UOL Economia no mês ...,"Vencer na vida\n\nProlongada pela ONU em 1948,...",120,80,20,20,0,"[120, 80, 20, 20, 0]"
452,Reportagem publicada pelo UOL Economia no mês ...,Vencer ou ser?\n\nVencer é algo que desde cedo...,160,160,120,160,120,"[160, 160, 120, 160, 120]"
453,Reportagem publicada pelo UOL Economia no mês ...,“E essa é a vitória que vence o mundo: a nossa...,160,120,160,160,120,"[160, 120, 160, 160, 120]"
454,Reportagem publicada pelo UOL Economia no mês ...,O que é mais importante para vencer na vida?\n...,20,80,20,20,20,"[20, 80, 20, 20, 20]"


In [3]:
df.to_csv("dataframe.csv")

In [20]:
from transformers import BertTokenizer, BertModel, LineByLineTextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict

tokenizer = BertTokenizer.from_pretrained('adalbertojunior/distilbert-portuguese-cased')
model = BertModel.from_pretrained("adalbertojunior/distilbert-portuguese-cased", output_attentions=True)#.to("cuda")

ds = Dataset.from_pandas(df)

train_testvalid = ds.train_test_split(test_size=0.2,seed=15)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5,seed=15)

# gather everyone if you want to have a single DatasetDict
data = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})


def tokenize(batch):
    return tokenizer(batch["essay"], truncation=True,max_length=512)

tokenized_dataset = data.map(tokenize, batched=True)
tokenized_dataset

Map:   0%|          | 0/364 [00:00<?, ? examples/s]

Map:   0%|          | 0/46 [00:00<?, ? examples/s]

Map:   0%|          | 0/46 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'essay', 'mark1', 'mark2', 'mark3', 'mark4', 'mark5', 'marks', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 364
    })
    test: Dataset({
        features: ['prompt', 'essay', 'mark1', 'mark2', 'mark3', 'mark4', 'mark5', 'marks', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 46
    })
    valid: Dataset({
        features: ['prompt', 'essay', 'mark1', 'mark2', 'mark3', 'mark4', 'mark5', 'marks', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 46
    })
})

In [21]:
from transformers import DataCollatorWithPadding

tokenized_dataset.set_format("torch",columns=["input_ids", "attention_mask", "marks"])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [37]:
import torch.nn.functional as F
import torch.nn as nn
import torch
from transformers.modeling_outputs import TokenClassifierOutput

class CustomModel(nn.Module):
    def __init__(self,num_labels): 
        super(CustomModel,self).__init__() 
        self.num_labels = num_labels 

        #Load Model with given checkpoint and extract its body
        self.model = model = BertModel.from_pretrained("adalbertojunior/distilbert-portuguese-cased",output_attentions=True,output_hidden_states=True)
        for param in model.parameters():
          param.requires_grad = False
        self.dropout = nn.Dropout(0.1) 
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        self.dropout3 = nn.Dropout(0.5)
        self.dropout4 = nn.Dropout(0.5)
        self.dropout5 = nn.Dropout(0.5)
        
        self.classifier1_mark1 = nn.Linear(768, 64) # Relates to the language domain
        self.classifier1_mark2 = nn.Linear(768, 64) # Relates to the understanding of the prompt
        self.classifier1_mark3 = nn.Linear(768, 64) # Relates to the argument organization
        self.classifier1_mark4 = nn.Linear(768, 64) # Relates to the liguistic mechanisms of argumentative texts
        self.classifier1_mark5 = nn.Linear(768, 64) # Relates to the solution elaboration

        self.classifier2_mark1 = nn.Linear(64, 1) # Relates to the language domain
        self.classifier2_mark2 = nn.Linear(64, 1) # Relates to the understanding of the prompt
        self.classifier2_mark3 = nn.Linear(64, 1) # Relates to the argument organization
        self.classifier2_mark4 = nn.Linear(64, 1) # Relates to the liguistic mechanisms of argumentative texts
        self.classifier2_mark5 = nn.Linear(64, 1) # Relates to the solution elaboration


    def forward(self, input_ids=None, attention_mask=None,marks=None):
        #Extract outputs from the body
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)

        #Add custom layers
        sequence_output = self.dropout(outputs[0]) #outputs[0]=last hidden state
        logit_mark1 = self.classifier1_mark1(sequence_output[:,0,:].view(-1,768))
        logit_mark2 = self.classifier1_mark2(sequence_output[:,0,:].view(-1,768))
        logit_mark3 = self.classifier1_mark3(sequence_output[:,0,:].view(-1,768))
        logit_mark4 = self.classifier1_mark4(sequence_output[:,0,:].view(-1,768))
        logit_mark5 = self.classifier1_mark5(sequence_output[:,0,:].view(-1,768))

        logit_mark1 = F.relu(logit_mark1)
        logit_mark2 = F.relu(logit_mark2)
        logit_mark3 = F.relu(logit_mark3)
        logit_mark4 = F.relu(logit_mark4)
        logit_mark5 = F.relu(logit_mark5)

        logit_mark1 = self.dropout1(logit_mark1)
        logit_mark2 = self.dropout2(logit_mark2)
        logit_mark3 = self.dropout3(logit_mark3)
        logit_mark4 = self.dropout4(logit_mark4)
        logit_mark5 = self.dropout5(logit_mark5)

        logit_mark1 = self.classifier2_mark1(logit_mark1)
        logit_mark2 = self.classifier2_mark2(logit_mark2)
        logit_mark3 = self.classifier2_mark3(logit_mark3)
        logit_mark4 = self.classifier2_mark4(logit_mark4)
        logit_mark5 = self.classifier2_mark5(logit_mark5)

        logits = torch.cat((logit_mark1, logit_mark2, logit_mark3, logit_mark4, logit_mark5), 1)

        loss1 = None
        loss2 = None
        loss3 = None
        loss4 = None
        loss5 = None
        if marks is not None:
            loss_fct = nn.MSELoss()
#            loss = loss_fct(logits.view(-1, self.num_labels), marks)
            loss1 = loss_fct(logit_mark1, marks[:,0].view(-1,1))
            loss2 = loss_fct(logit_mark2, marks[:,1].view(-1,1))
            loss3 = loss_fct(logit_mark3, marks[:,2].view(-1,1))
            loss4 = loss_fct(logit_mark4, marks[:,3].view(-1,1))
            loss5 = loss_fct(logit_mark5, marks[:,4].view(-1,1))

        return dict(loss1=loss1, loss2=loss2, loss3=loss3, loss4=loss4, loss5=loss5,
                    logits=logits, hidden_states=outputs.hidden_states,attentions=outputs.attentions)

In [38]:
import torch 
device = torch.device("cpu")#("cuda" if torch.cuda.is_available() else "cpu")
model=CustomModel(num_labels=5).to(device)

from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_dataset["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_dataset["valid"], batch_size=8, collate_fn=data_collator
)

In [39]:
from transformers import get_scheduler
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

138


In [40]:
from evaluate import load
metric = load("f1")

In [41]:
from tqdm.auto import tqdm
#import os
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "garbage_collection_threshold:0.6,max_split_size_mb:128"

progress_bar_train = tqdm(range(num_training_steps))
progress_bar_eval = tqdm(range(num_epochs * len(eval_dataloader)))
metrics = []

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)

        loss1 = outputs['loss1']
        loss2 = outputs['loss2']
        loss3 = outputs['loss3']
        loss4 = outputs['loss4']
        loss5 = outputs['loss5']

        loss = loss1 + loss2 + loss3 + loss4 + loss5
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar_train.update(1)

    model.eval()
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs['logits']
        metric.update(preds=logits, target=batch["marks"])
        progress_bar_eval.update(1)

    metrics.append(metric.compute())

  0%|          | 0/138 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

torch.Size([8, 512, 768])
torch.Size([8, 5])


RuntimeError: Expected floating point type for target with class probabilities, got Long

In [12]:
import gc
def report_gpu():
    #print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()

In [18]:
report_gpu()

In [ ]:
import matplotlib.pyplot as plt
cpu_metrics = torch.tensor(metrics, device = 'cpu')
plt.plot(cpu_metrics)
plt.show()